<a href="https://colab.research.google.com/github/morfant/TF_study/blob/master/TF_week_01_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import numpy as np
import tensorflow as tf
!pip install -q tensorflow-hub
!pip install -q tfds-nightly
import tensorflow_hub as hub
import tensorflow_datasets as tfds

print('버전: ', tf.__version__)
print('즉시 실행 모드: ', tf.executing_eagerly())
print('허브 버전: ', hub.__version__)
print('GPU ', '사용 가능' if tf.config.experimental.list_physical_devices("GPU") else '사용 불가능')


     |████████████████████████████████| 3.3MB 2.9MB/s 
버전:  2.2.0-rc2
즉시 실행 모드:  True
허브 버전:  0.8.0
GPU  사용 불가능


In [0]:

train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews", 
    split=('train[:60%]',  'train[60%:]', 'test'),
    as_supervised=True)


In [12]:
train_example_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_example_batch
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0])>

In [14]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], dtype=tf.string, trainable=True)
hub_layer(train_example_batch[:3])

<tf.Tensor: shape=(3, 20), dtype=float32, numpy=
array([[ 1.765786  , -3.882232  ,  3.9134233 , -1.5557289 , -3.3362343 ,
        -1.7357955 , -1.9954445 ,  1.2989551 ,  5.081598  , -1.1041286 ,
        -2.0503852 , -0.72675157, -0.65675956,  0.24436149, -3.7208383 ,
         2.0954835 ,  2.2969332 , -2.0689783 , -2.9489717 , -1.1315987 ],
       [ 1.8804485 , -2.5852382 ,  3.4066997 ,  1.0982676 , -4.056685  ,
        -4.891284  , -2.785554  ,  1.3874227 ,  3.8476458 , -0.9256538 ,
        -1.896706  ,  1.2113281 ,  0.11474707,  0.76209456, -4.8791065 ,
         2.906149  ,  4.7087674 , -2.3652055 , -3.5015898 , -1.6390051 ],
       [ 0.71152234, -0.6353217 ,  1.7385626 , -1.1168286 , -0.5451594 ,
        -1.1808156 ,  0.09504455,  1.4653089 ,  0.66059524,  0.79308075,
        -2.2268345 ,  0.07446612, -1.4075904 , -0.70645386, -1.907037  ,
         1.4419787 ,  1.9551861 , -0.42660055, -2.8022065 ,  0.43727064]],
      dtype=float32)>

In [15]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_1 (KerasLayer)   (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [17]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=20,
                    validation_data=validation_data.batch(512),
                    verbose=1)

Epoch 1/20
30/30 [==============================] - 3s 108ms/step - loss: 0.7420 - accuracy: 0.5176 - val_loss: 0.6877 - val_accuracy: 0.5606
Epoch 2/20
30/30 [==============================] - 3s 99ms/step - loss: 0.6446 - accuracy: 0.5994 - val_loss: 0.6252 - val_accuracy: 0.6212
Epoch 3/20
30/30 [==============================] - 3s 101ms/step - loss: 0.5924 - accuracy: 0.6517 - val_loss: 0.5834 - val_accuracy: 0.6586
Epoch 4/20
30/30 [==============================] - 3s 101ms/step - loss: 0.5518 - accuracy: 0.6938 - val_loss: 0.5465 - val_accuracy: 0.7067
Epoch 5/20
30/30 [==============================] - 3s 101ms/step - loss: 0.5119 - accuracy: 0.7299 - val_loss: 0.5124 - val_accuracy: 0.7211
Epoch 6/20
30/30 [==============================] - 3s 103ms/step - loss: 0.4757 - accuracy: 0.7566 - val_loss: 0.4786 - val_accuracy: 0.7646
Epoch 7/20
30/30 [==============================] - 3s 102ms/step - loss: 0.4376 - accuracy: 0.7917 - val_loss: 0.4480 - val_accuracy: 0.7860
Epoch 8

In [20]:
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

49/49 - 3s - loss: 0.3211 - accuracy: 0.8575
loss: 0.321
accuracy: 0.857
